В домашньому завданні до даного модулю ви потренуєтесь робити тестове завдання для влаштування на роботу. За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Знайти датасет ви можете за https://drive.google.com/file/d/1nzrtQpfaHL0OgJ_eXzA7VuEj7XotrSWO/view.

Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Більше ці характеристики описані в даній https://drive.google.com/file/d/1-18YEmp0YjV3hN9iI8J1i_FWd55HFwOK/view.

Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою.

Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою. Використайте метод classification report для порівняння.

Порівняння моделей на основі однієї метрики(такої як Accuracy)- не приймається.

In [1]:
print ('Завантаження даних')

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Шлях до папки з даними
data_path = "data"

# Функція для зчитування всіх CSV файлів з папки
def load_data(data_path):
    activities = ['idle', 'running', 'stairs', 'walking']
    data = []
    labels = []
    for activity in activities:
        activity_path = os.path.join(data_path, activity)
        for file_name in os.listdir(activity_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(activity_path, file_name)
                df = pd.read_csv(file_path)
                data.append(df)
                labels.append(activity)
    return data, labels

data, labels = load_data(data_path)

Завантаження даних


In [2]:
print ('\nОбробка даних')

# Поєднання всіх даних в один DataFrame
combined_data = pd.concat(data, ignore_index=True)
combined_data['label'] = np.repeat(labels, [len(df) for df in data])


Обробка даних


In [3]:
print ('\nОбчислення часових ознак')

def extract_features(df):
    features = {}
    features['mean_x'] = df['accelerometer_X'].mean()
    features['mean_y'] = df['accelerometer_Y'].mean()
    features['mean_z'] = df['accelerometer_Z'].mean()
    features['std_x'] = df['accelerometer_X'].std()
    features['std_y'] = df['accelerometer_Y'].std()
    features['std_z'] = df['accelerometer_Z'].std()
    features['max_x'] = df['accelerometer_X'].max()
    features['max_y'] = df['accelerometer_Y'].max()
    features['max_z'] = df['accelerometer_Z'].max()
    features['min_x'] = df['accelerometer_X'].min()
    features['min_y'] = df['accelerometer_Y'].min()
    features['min_z'] = df['accelerometer_Z'].min()
    features['median_x'] = df['accelerometer_X'].median()
    features['median_y'] = df['accelerometer_Y'].median()
    features['median_z'] = df['accelerometer_Z'].median()
    return features

# Обчислення ознак для кожного з файлів
features_list = [extract_features(df) for df in data]
features_df = pd.DataFrame(features_list)
features_df['label'] = labels


Обчислення часових ознак


In [4]:
print ('\nПідготовка вибірок для тестування та тренування')

X = features_df.drop('label', axis=1)
y = features_df['label']

# Розділення даних на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Нормалізація даних
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Підготовка вибірок для тестування та тренування


In [5]:
print ('\nТренування моделей SVM та Random Forest')

# Тренування SVM
svm = SVC()
svm.fit(X_train, y_train)

# Тренування Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)


Тренування моделей SVM та Random Forest


RandomForestClassifier()

In [6]:
print ('\nОцінка моделей')

# Прогнозування на тестових даних
y_pred_svm = svm.predict(X_test)
y_pred_rf = rf.predict(X_test)

# Оцінка моделей
report_svm = classification_report(y_test, y_pred_svm)
report_rf = classification_report(y_test, y_pred_rf)

print("SVM Classification Report:\n", report_svm)
print("Random Forest Classification Report:\n", report_rf)


Оцінка моделей
SVM Classification Report:
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00       317
     running       1.00      1.00      1.00      1035
      stairs       0.87      0.68      0.76        50
     walking       0.97      0.99      0.98       537

    accuracy                           0.99      1939
   macro avg       0.96      0.92      0.94      1939
weighted avg       0.99      0.99      0.99      1939

Random Forest Classification Report:
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00       317
     running       1.00      1.00      1.00      1035
      stairs       0.98      0.98      0.98        50
     walking       1.00      1.00      1.00       537

    accuracy                           1.00      1939
   macro avg       0.99      0.99      0.99      1939
weighted avg       1.00      1.00      1.00      1939



In [7]:
print ('\nПорівняння результатів')

# Друк звітів
print("SVM Classification Report:\n", report_svm)
print("Random Forest Classification Report:\n", report_rf)


Порівняння результатів
SVM Classification Report:
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00       317
     running       1.00      1.00      1.00      1035
      stairs       0.87      0.68      0.76        50
     walking       0.97      0.99      0.98       537

    accuracy                           0.99      1939
   macro avg       0.96      0.92      0.94      1939
weighted avg       0.99      0.99      0.99      1939

Random Forest Classification Report:
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00       317
     running       1.00      1.00      1.00      1035
      stairs       0.98      0.98      0.98        50
     walking       1.00      1.00      1.00       537

    accuracy                           1.00      1939
   macro avg       0.99      0.99      0.99      1939
weighted avg       1.00      1.00      1.00      1939

